# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
cities_df.drop(["a"], axis=1, inplace=True)
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,busselton,66,AU,1580357043,31,-33.65,115.33,73.99,15.28
1,trelleborg,90,SE,1580357724,93,55.38,13.16,39.99,16.11
2,rikitea,100,PF,1580357724,72,-23.12,-134.97,79.39,6.96
3,cabo san lucas,75,MX,1580357707,59,22.89,-109.91,66.20,4.70
4,qaanaaq,100,GL,1580357706,95,77.48,-69.36,-19.32,1.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
gmaps.figure()
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]
humidity

0      31
1      93
2      72
3      59
4      95
       ..
565    82
566    74
567    94
568    83
569    69
Name: Humidity, Length: 570, dtype: int64

In [4]:
Heatmap = gmaps.figure()
 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=True, max_intensity=100, point_radius = 10)
Heatmap.add_layer(heat_layer)

Heatmap


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_city_df = cities_df.loc[(cities_df['Cloudiness']<25) & (cities_df['Humidity']>25) & (cities_df['Humidity']<75) & 
                           (cities_df['Max Temp']>50) & (cities_df['Max Temp']<100) & (cities_df['Wind Speed']<50)].reset_index(drop=True)
new_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dicabisagan,19,PH,1580357726,63,17.08,122.42,77.49,6.04
1,saint-philippe,17,RE,1580357726,57,-21.36,55.77,80.60,3.36
2,punta arenas,20,CL,1580357728,71,-53.15,-70.92,51.80,32.21
3,usta muhammad,0,PK,1580357728,63,28.18,68.04,52.48,4.12
4,laem sing,20,TH,1580357729,62,12.48,102.07,90.00,10.29
...,...,...,...,...,...,...,...,...,...
56,tateyama,20,JP,1580357781,42,34.98,139.87,62.60,12.75
57,iquique,0,CL,1580357521,60,-20.22,-70.14,75.20,10.29
58,nandu,0,CN,1580357783,50,22.85,110.82,63.00,1.01
59,ongole,0,IN,1580357784,74,15.50,80.05,76.10,3.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_city_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,dicabisagan,19,PH,1580357726,63,17.08,122.42,77.49,6.04,
1,saint-philippe,17,RE,1580357726,57,-21.36,55.77,80.60,3.36,
2,punta arenas,20,CL,1580357728,71,-53.15,-70.92,51.80,32.21,
3,usta muhammad,0,PK,1580357728,63,28.18,68.04,52.48,4.12,
4,laem sing,20,TH,1580357729,62,12.48,102.07,90.00,10.29,
...,...,...,...,...,...,...,...,...,...,...
56,tateyama,20,JP,1580357781,42,34.98,139.87,62.60,12.75,
57,iquique,0,CL,1580357521,60,-20.22,-70.14,75.20,10.29,
58,nandu,0,CN,1580357783,50,22.85,110.82,63.00,1.01,
59,ongole,0,IN,1580357784,74,15.50,80.05,76.10,3.83,


In [7]:
params = {"radius" : 50000,
    "types": "Hotel",
    "keyword": "Lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (IndexError):
        print("Hotel not found")
          
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,dicabisagan,19,PH,1580357726,63,17.08,122.42,77.49,6.04,Umengan Clan
1,saint-philippe,17,RE,1580357726,57,-21.36,55.77,80.60,3.36,Les Lodges Vanilliers
2,punta arenas,20,CL,1580357728,71,-53.15,-70.92,51.80,32.21,Alexander
3,usta muhammad,0,PK,1580357728,63,28.18,68.04,52.48,4.12,Mukhtiar Ahmed manjhoo
4,laem sing,20,TH,1580357729,62,12.48,102.07,90.00,10.29,Chaolao Cabana Resort
...,...,...,...,...,...,...,...,...,...,...
56,tateyama,20,JP,1580357781,42,34.98,139.87,62.60,12.75,Kinokuniya Ryokan
57,iquique,0,CL,1580357521,60,-20.22,-70.14,75.20,10.29,Hotel estancia
58,nandu,0,CN,1580357783,50,22.85,110.82,63.00,1.01,Kunlun International Hotel
59,ongole,0,IN,1580357784,74,15.50,80.05,76.10,3.83,Seabreeze Beach Resorts


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
Heatmap.add_layer(markers)

# Display Map
Heatmap

Figure(layout=FigureLayout(height='420px'))